In [1]:
import sys
from os import listdir, path
import MeCab
import re
from gensim import models
from gensim.models.doc2vec import LabeledSentence
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [144]:
#聖書よむ
with open("bibletext.txt", "rU") as f:
    text = [v.rstrip() for v in f.readlines()]
    
#先頭が空欄だったので削除
text.pop(0)

#いらない先頭の数字を削除
text_no_numbers=[]
for i in range(0,len(text)):
    text_no_number=re.sub("^\d+.\d+\s", "",text[i])
    text_no_numbers.append(text_no_number)

/Users/test/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 'U' mode is deprecated
  


In [151]:
#形態素解析。名詞、動詞とかだけ残して、単語をカンマで区切る処理。

def wakachi(text):
    tagger = MeCab.Tagger('-Ochasen')
    tagger.parse('') # <= 空文字列をparseする
    tagger.parseToNode('')

    word_classes = [u'名詞',u'動詞',u'形容詞',u'形容動詞',u'副詞']#使いたい文字を指定
    keywords = []
    
    for i in range(0,len(text)):
        keyword=[]
        node = tagger.parseToNode(text[i])
        
        while node:
            if node.feature.split(",")[0] in word_classes:
                keyword.append(node.surface)
            node = node.next
        keywords.append(keyword)
    return keywords

In [146]:
#質問文と一番近い聖書のテキストIDを出す。

def search_similar_texts(question):
    x = m.infer_vector(question)
    most_similar_texts = m.docvecs.most_similar([x])
    for similar_text in most_similar_texts:
        return(similar_text[0])

In [152]:
#１文書ずつ、単語に分割してリストに入れていく[([単語1,単語2,単語3],文書id),...]こんなイメージ
#words：文書に含まれる単語のリスト（単語の重複あり）
#tags：文書の識別子（リストで指定．1つの文書に複数のタグを付与できる）

keywords=wakachi(text_no_numbers)
trainings = [TaggedDocument(words =data, tags = [i]) for (i,data) in enumerate(keywords)]



In [2]:
#モデルのセーブ
m.save("doc2vec.model")

# モデルのロード(モデルが用意してあれば、ここからで良い)
m = Doc2Vec.load('doc2vec.model')

In [175]:
#トレーニング
m = Doc2Vec(documents= trainings, dm = 1, size=1000, window=8, min_count=100, workers=1)

In [176]:
question =["先日、息子から結婚すると報告があり、喜んでいた矢先…実は息子の結婚相手は2年程前に息子とお付き合いしていた元恋人で、別れた後に彼女から妊娠を告げられたらしく、息子はやり直すつもりは無く、妊娠の事実を知りながらも何もしなかった。と告げられました。そして去年の秋頃から、元恋人とよりを戻し、新たな命を授かったので結婚を決意したと言われました。彼女は1人で子育てし、現在1歳の男の子の写メールを見たのですが、息子にそっくりで、彼女への申し訳なさが溢れました。ですが私の夫は、息子の行動に激怒し、出て行けと、息子を突き放しました。私は、彼女に会って謝りたいと同時に仲良くしていきたいです。彼女のご両親は、結婚に猛反対だそうです。どうして今頃、また妊娠したからって結婚はないんじゃないか、、と。1歳の子の1年間の責任はどうするつもりだ。と言われてるそうです。相手のご両親のお気持ちもわかりますし、20歳超えた大人の行動として恥ずかしい息子だと思います。彼女、相手のご両親に謝罪に行くべきなのでしょうか。"]
question=wakachi(question)
print(question)

sim_id=search_similar_texts(question[0])
print(sim_id,text_no_numbers[sim_id])

[['先日', '息子', '結婚', 'する', '報告', 'あり', '喜ん', 'い', '矢先', '息子', '結婚', '相手', '2', '年', '前', '息子', '付き合い', 'し', 'い', '恋人', '別れ', '後', '彼女', '妊娠', '告げ', 'られ', '息子', 'やり直す', 'つもり', '無く', '妊娠', '事実', '知り', '何', 'し', '告げ', 'られ', '去年', '秋', '頃', '恋人', 'より', '戻し', '新た', '命', '授かっ', '結婚', '決意', 'し', '言わ', 'れ', '彼女', '1', '人', '子育て', 'し', '現在', '1', '歳', '男の子', '写', 'メール', '見', 'の', '息子', 'そっくり', '彼女', '申し訳', 'さ', '溢れ', '私', '夫', '息子', '行動', '激怒', 'し', '出', '行け', '息子', '突き放し', '私', '彼女', '会っ', '謝り', '同時に', '仲良く', 'し', 'いき', '彼女', '両親', '結婚', '反対', 'そう', 'どうして', '今頃', '妊娠', 'し', '結婚', 'ない', 'ん', '1', '歳', '子', '1', '年間', '責任', 'どう', 'する', 'つもり', '言わ', 'れ', 'てる', 'そう', '相手', '両親', '気持ち', 'わかり', '20', '歳', '超え', '大人', '行動', '恥ずかしい', '息子', '思い', '彼女', '相手', '両親', '謝罪', '行く', 'の']]
7 あなた方は彼を見たことがありませんが，愛しています。今は彼を見ることができませんが，それでも信じており，言葉に表せない，栄光に満ちた喜びをもって大いに喜んでいます。


In [177]:
question =["神が信じられません。神はなぜ人間同士の争いをそのままにしておくのでしょうか。他の信仰があるものは他の信仰を持っていればいいもおのを、なぜ神はそれを許さず、殺させるのでしょうか。一人の神への信仰と一人の人間の苦しみ、どちらが重いかを考えた時、後者の方が重いと思うのです。こんなことを言うと、神はお怒りになるかもしれませんが、そんな度量の小さい神はいりません。"]
question=wakachi(question)
print(question)

sim_id=search_similar_texts(question[0])
print(sim_id,text_no_numbers[sim_id])

[['神', '信じ', 'られ', '神', 'なぜ', '人間', '同士', '争い', 'そのまま', 'し', 'おく', 'の', '他', '信仰', 'ある', 'もの', '他', '信仰', '持っ', 'いれ', 'いい', 'おの', 'なぜ', '神', 'それ', '許さ', '殺さ', 'せる', 'の', '一', '人', '神', '信仰', '一', '人', '人間', '苦しみ', 'どちら', '重い', '考え', '時', '後者', '方', '重い', '思う', 'の', 'こと', '言う', '神', '怒り', 'なる', 'しれ', '度量', '小さい', '神', 'はいり']]
29 あなた方も，生きた石となって，聖なる祭司職のため，また，イエス・キリストを通して神に受け入れられる霊的な犠牲をささげるための霊的な家として築き上げられなさい。


In [178]:
question =["彼氏が海外転勤のため別れました。彼か私どちらがつらいのでしょうか？彼氏7月から欧州に最長5年間転勤になりました。まだ付き合って半年でした。彼氏から海外転勤の辞令が出たと電話で聞いたときは、以前から海外転勤が夢だと知っていたので泣きながら「よかったね！おめでとう！」と言いました。彼はそれを聞いて号泣していました。その日の電話で4時間ほど話、これから出国までの3ヶ月お互いどのようにして付き合いたいか（どこに行きたいかとかお互いにしてほしいこととか良い方向の）話をしたりとても良い雰囲気でした。彼氏は5年間日本と海外での遠距離を希望していました。私もあまりにも突然の話だったので、何も考えられず「遠距離は自信ないけど・・」と言いながらも別れるなんて考えられなかったので同意しました。それにお互い二十代後半なのでもちろん将来も視野にいれて遠距離を提案しているのだとばかり思っていました。しかし、電話の翌日、両親に彼氏が遠距離になり5年間遠距離すると話すと「結婚はどうする気なの？何の当てもなく5年も日本と海外で遠距離するの？5年たてば30歳になってるのに」と言われ不安になって彼に直ぐに電話して聞くと、「現時点で結婚してついてきてとは言えないし、何年後かに呼ぶと言う方法もあるけどいつ呼べる状況になるかも分からない。だから今はただ遠距離するか、別れるかの2つしか選択しはない。それはそっちが選んで」と言われました。あまりにも彼の口から出てくる言葉とは思えずショックで、電話を切って直ぐにこちらから別れのメールお送りました。しかし、一時の感情で送ってしまったのでそれから2度会って話す機会を私から作ってもらいました。今はとにかく好きなので別れたくないし遠距離をしたいと思ったからです。でも彼氏は別れるの一点張りです。その理由は「これから3ヶ月付き合えばまた思い出が増えて、今よりもっと好きになる。出国する時、さらに離れるのがつらくなる。良く考えれば現実問題5年間の遠距離は無理だと思う。それなら今お互いのために別れるのが一番いい」と言うものでした。彼は電話でも、会って話しても毎回、私よりも号泣です。最後に別れた時もずっと泣きっぱなしでした。自分の都合で別れると言っているのになぜ彼のほうが泣くのですか？一方的な別れを告げられた私のほうがつらいと思うのですが・・・・補足"]
question=wakachi(question)
print(question)

sim_id=search_similar_texts(question[0])
print(sim_id,text_no_numbers[sim_id])

[['彼氏', '海外', '転勤', 'ため', '別れ', '彼', '私', 'どちら', 'つらい', 'の', '彼氏', '7', '月', '欧州', '最長', '5', '年間', '転勤', 'なり', 'まだ', '付き合っ', '半年', '彼氏', '海外', '転勤', '辞令', '出', '電話', '聞い', 'とき', '以前', '海外', '転勤', '夢', '知っ', 'い', '泣き', 'よかっ', '言い', '彼', 'それ', '聞い', '号泣', 'し', 'い', '日', '電話', '4', '時間', '話', 'これから', '出国', '3', 'ヶ月', 'お互い', 'よう', 'し', '付き合い', 'どこ', '行き', 'お互い', 'し', 'ほしい', 'こと', '良い', '方向', '話', 'し', 'とても', '良い', '雰囲気', '彼氏', '5', '年間', '日本', '海外', '遠距離', '希望', 'し', 'い', '私', 'あまりに', '突然', '話', '何', '考え', 'られ', '遠距離', '自信', 'ない', '言い', '別れる', '考え', 'られ', '同意', 'し', 'それ', 'お互い', '二', '十', '代', '後半', 'もちろん', '将来', '視野', 'いれ', '遠距離', '提案', 'し', 'いる', 'の', '思っ', 'い', '電話', '翌日', '両親', '彼氏', '遠距離', 'なり', '5', '年間', '遠距離', 'する', '話す', '結婚', 'どう', 'する', '気', '何', '当て', 'なく', '5', '年', '日本', '海外', '遠距離', 'する', '5', '年', 'たて', '30', '歳', 'なっ', 'てる', '言わ', 'れ', '不安', 'なっ', '彼', '直ぐ', '電話', 'し', '聞く', '現時点', '結婚', 'し', 'つい', 'き', '言え', '何', '年', '後', '呼ぶ', '言う', '方法', 'ある', 'いつ', '呼べる', '状況', 'なる',

In [179]:
question =["家にいるのが辛いです。私は高校一年生です。父親と母親はとても仲が悪く仮面夫婦です。原因は三四つあると思います。まず、母親が過去に浮気をしていたことです。なぜ私が知ったのかは覚えていません。でも、その事で私は軽い人間不信のようになりました。そして、今まで大好きだった母親が嫌いになりました。話がそれてしまいましたが、父もその事で母を信じられなくなったそうです。私は母は今でも浮気をしていると思っています。その事についてはもう諦めていますがやっぱり嫌です、そんな母親。二つ目は、母がアルコール依存であることです。私も父も何度も注意しました。しかし、反省はその場限りでいつまでたっても治りません。そのせいでどんどん家事もおろそかになってきている気がします。三つ目は、母親の親族と父親の親族が仲が悪いことです。考え方や家族の雰囲気も正反対な感じで、私から見ても相性は最悪だと思います。四つ目は、父が極端に短気なことです。母の浮気以来、父は母を見下しています。ですが、私から見ても異常なほどよくキレます。父は元ヤンなので正直とても怖いです。そしてその度に、妹2人(小六と年長)は怯えて号泣します。その時に面倒を見るのもすべて私です。もう辛いです。別居や、離婚もあるとは思いますが、そう簡単にはできないし、何より妹たちは両親が大好きなのでそんなことできません。私も昔は家族が大好きでした。ですが、今では家にいるのも辛いです。現に昨日も両親は喧嘩をして、今日も母は帰ってこないそうです。また浮気でもしているのではと思います。もう円満になってほしいなんて望んでいません。でも、もう今にも心が壊れそうです。昨日の喧嘩のあとも一人で過呼吸になり叫んでしまいました。何を言ったかは覚えていませんが、｢みんないなくなればいい。こんな家出ていってやる。｣というような事だった気がします。(もちろん妹には聞こえないようにしました。)でも、普段は家族にこういう感情がバレないように明るく振る舞っています。母も父も私たちのことを自慢するのが好きなので、勉強や部活も頑張っています。そういうところはまだ両親が好きなのかなともおもいます。でも、辛いです。私も友達のように明るく楽しく遊んだりしたいです。普通の家庭で育ち、普通の高校生活が送れたらそれでいいんです。私はどうすればいいのでしょうか。"]
question=wakachi(question)
print(question)

sim_id=search_similar_texts(question[0])
print(sim_id,text_no_numbers[sim_id])

[['家', 'いる', 'の', '辛い', '私', '高校', '一', '年生', '父親', '母親', 'とても', '仲', '悪く', '仮面', '夫婦', '原因', '三', '四つ', 'ある', '思い', 'まず', '母親', '過去', '浮気', 'し', 'い', 'こと', 'なぜ', '私', '知っ', 'の', '覚え', 'い', '事', '私', '軽い', '人間', '不信', 'よう', 'なり', '今', '大好き', '母親', '嫌い', 'なり', '話', 'それ', 'しまい', '父', '事', '母', '信じ', 'られ', 'なっ', 'そう', '私', '母', '今', '浮気', 'し', 'いる', '思っ', 'い', '事', 'もう', '諦め', 'い', 'やっぱり', '嫌', '母親', '二つ', '目', '母', 'アルコール', '依存', 'こと', '私', '父', '何', '度', '注意', 'し', '反省', '場', '限り', 'いつ', 'たっ', '治り', 'せい', 'どんどん', '家事', 'おろそか', 'なっ', 'き', 'いる', '気', 'し', '三つ', '目', '母親', '親族', '父親', '親族', '仲', '悪い', 'こと', '考え方', '家族', '雰囲気', '正反対', '感じ', '私', '見', '相性', '最悪', '思い', '四つ', '目', '父', '極端', '短気', 'こと', '母', '浮気', '以来', '父', '母', '見下し', 'い', '私', '見', '異常', 'よく', 'キレ', '父', 'ヤン', '正直', 'とても', '怖い', '度', '妹', '2', '人', '(', '小', '六', '年長', ')', '怯え', '号泣', 'し', '時', '面倒', '見る', 'の', 'すべて', '私', 'もう', '辛い', '別居', '離婚', 'ある', '思い', 'そう', '簡単', 'でき', '何より', '妹', 'たち', '両親', '大好き', 'こと', 'でき', '私'

以下よりツイッターの部

In [130]:
import tweepy
from requests_oauthlib import OAuth1Session
import json

CK = ''                             # Consumer Key
CS = ''         # Consumer Secret
auth = tweepy.OAuthHandler(CK, CS)
AT = '' # Access Token
AS = ''# Accesss Token Secert
auth.set_access_token(AT, AS)

api = tweepy.API(auth)


In [119]:
# ツイート投稿用のURL
url = "https://api.twitter.com/1.1/statuses/update.json"

# ツイート本文
params = {"status": "テスト"}

# OAuth認証で POST method で投稿
twitter = OAuth1Session(CK, CS, AT, AS)
req = twitter.post(url, params = params)

# レスポンスを確認
if req.status_code == 200:
    print ("OK")
else:
    print ("Error: %d" % req.status_code)

OK


In [ ]:
#自動でつぶやく

In [134]:
#返信につぶやく
#参考ページ：http://net.univ-q.com/archives/38

if str(status.in_reply_to_screen_name)=="{Bot_TWITTER_ID}":
    tweet = "@" + str(status.user.screen_name) + "answer" 
    api.update_status(status=tweet)


NameError: name 'status' is not defined

timeline=api.mentions_timeline()
for status in timeline:
　　　status_id=status.id
　　　screen_name=status.author.screen_name.encode("UTF-8")
　　　reply_text="@"+screen_name+" "+"リプライテキスト"
　　　api.update_status(status=reply_text,in_reply_to_status_id=status_id)